In [17]:
from msedge.selenium_tools import Edge, EdgeOptions
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re

In [8]:
options = EdgeOptions()
options.use_chromium = True
options.add_argument("headless")
driver = Edge(executable_path=r'C:\dev\WebDrivers\edgedriver_win64_111\msedgedriver.exe',options=options)

In [9]:
driver.get('https://suumo.jp/ms/chuko/fukuoka/city/')

In [13]:
search_box = driver.find_element(By.CLASS_NAME, 'searchbox')

In [14]:
search_button = search_box.find_elements_by_tag_name("a")

In [15]:
search_button

<selenium.webdriver.remote.webelement.WebElement (session="ac070ab71d31fef15f1aa011168f2bb4", element="ef82a263-2e31-49d2-bdc9-f8e4d98461af")>

In [16]:
search_button.click()

In [26]:
properties = driver.find_elements(By.XPATH, "//div[starts-with(@class,'property_unit')]")

In [27]:
properties

[<selenium.webdriver.remote.webelement.WebElement (session="ac070ab71d31fef15f1aa011168f2bb4", element="4a473aba-9975-4502-9f27-bde409f93480")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ac070ab71d31fef15f1aa011168f2bb4", element="6caf7674-3ed3-450c-be7d-9fea84696171")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ac070ab71d31fef15f1aa011168f2bb4", element="19cd1e4f-2e32-463c-a1da-2920ecee97d2")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ac070ab71d31fef15f1aa011168f2bb4", element="a926736b-6b42-47af-a649-7680770ba7d8")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ac070ab71d31fef15f1aa011168f2bb4", element="a233dea8-d045-4118-b2ae-16344331572e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ac070ab71d31fef15f1aa011168f2bb4", element="3e6825f3-57d4-48b7-bbe4-4cc7be4d73fa")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ac070ab71d31fef15f1aa011168f2bb4", element="d51c2d8d-f1b2-46e3-99f2-8d

In [28]:
price = properties[0].find_element(By.CLASS_NAME, 'dottable-value')

In [30]:
price.text

'250万円'